In [1]:
import pandas as pd 
training = pd.read_csv('working_best.csv')
display(training)

,Job Id,Experience,Qualifications,Salary Range,Work Type,Job Posting Date,Job Title,Role,Job Portal,Job Description,Benefits,Responsibilities,Company,Company Profile,tokenized_skills
0,1089843540111562,5 to 15 years,['mtech'],59-99k,['intern'],2022-04-24,['digital marketing specialist'],['social media manager'],['snagajob'],['social media managers oversee an organizatio...,flexible spending accounts fsas relocation ass...,manage and grow social media accounts create e...,['icahn enterprises'],diversified industry diversified financialscit...,"['social media platforms eg', 'content creatio..."
1,398454096642776,2 to 12 years,['bca'],56-116k,['intern'],2022-12-19,['web developer'],['frontend web developer'],['idealist'],['frontend web developers design and implement...,health insurance retirement plans paid time of...,design and code user interfaces for websites e...,['pnc financial services group'],financial services industry commercial banksci...,"['html', 'css', 'javascript', 'frontend framew..."
2,481640072963533,0 to 12 years,['phd'],61-104k,['temporary'],2022-09-14,['operations manager'],['quality control manager'],['jobs2careers'],['quality control managers establish and enfor...,legal assistance bonuses and incentive program...,establish and enforce quality control standard...,['united services automobile assn'],insurance industry insurance property and casu...,['quality control processes and methodologies'...
3,688192671473044,4 to 11 years,['phd'],65-91k,['fulltime'],2023-02-25,['network engineer'],['wireless network engineer'],['flexjobs'],['wireless network engineers design implement ...,transportation benefits professional developme...,design configure and optimize wireless network...,['hess'],energy industry mining crudeoil productioncity...,"['wireless network design and architecture', '..."
4,117057806156508,1 to 12 years,['mba'],64-87k,['intern'],2022-10-11,['event manager'],['conference manager'],['jobs2careers'],['a conference manager coordinates and manages...,flexible spending accounts fsas relocation ass...,specialize in conference and convention planni...,['cairn energy'],energy industry energy oil gas exploration ...,"['event planning', 'conference logistics', 'bu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107359,1983179143713411,3 to 12 years,['bba'],60-128k,['parttime'],2021-11-02,['chemical engineer'],['research and development rd engineer'],['indeed'],['conduct research design experiments and deve...,health insurance retirement plans flexible wor...,work on research and development of new produc...,['the goldman sachs group inc'],financial servicesinvestment banking industry ...,"['research and development processes', 'produc..."
107360,669857235415970,2 to 15 years,['mba'],59-121k,['intern'],2023-07-01,['uxui designer'],['user interface designer'],['indeed'],['user interface designers focus on the visual...,employee assistance programs eap tuition reimb...,create visually appealing user interfaces ui t...,['thrivent financial for lutherans'],financial services industry insurance life hea...,"['ui design principles and best practices', 'g..."
107361,2722200185343244,0 to 11 years,['mca'],59-80k,['intern'],2023-05-18,['account director'],['account strategist'],['snagajob'],['account strategists work in advertising or m...,life and disability insurance stock options or...,develop account strategies and marketing plans...,['downer group'],constructioninfrastructure industry constructi...,"['account management', 'client relations', 'ma..."
107362,2740245700222760,1 to 9 years,['bba'],63-83k,['fulltime'],2023-04-18,['nurse practitioner'],['acute care nurse practitioner'],['flexjobs'],['acute care nurse practitioners focus on mana...,tuition reimbursement stock options or equity ...,specialize in acute care settings managing com...,['oreilly automotive'],automotive industry specialty retailers otherc...,"['acute care', 'nursing emergency', 'medicine'..."


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pandas as pd


# Assuming you have a DataFrame 'data' with job data
data = pd.read_csv('working_best.csv')

# Feature extraction using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['Job Title'])

# Train KMeans model
n_clusters = 3  # Number of clusters for high, medium, and low demand
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(X)



# Assign cluster labels to job data
data['cluster'] = kmeans.labels_

# Group job titles by cluster
clusters = {}
for cluster_label in range(n_clusters):
    cluster_data = data[data['cluster'] == cluster_label]
    job_titles = set()
    for job_title in cluster_data['Job Title']:
        if job_title not in job_titles:
            job_titles.add(job_title)
            if len(job_titles) == 5:  # Limit to first 5 unique jobs
                break
    clusters[cluster_label] = job_titles

# Display job titles for each cluster
for cluster_label, job_titles in clusters.items():
    print(f'Cluster {cluster_label}:')
    for job_title in job_titles:
        print(job_title)
    print('\n')


/home/dell/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Cluster 0:
['teacher']
['network engineer']
['web developer']
['software tester']
['digital marketing specialist']


Cluster 1:
['operations manager']
['event manager']
['account manager']
['brand manager']
['sales manager']


Cluster 2:
['landscape designer']
['architectural designer']
['electrical designer']
['interior designer']
['uxui designer']




In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pandas as pd
import joblib 

# Assuming you have a DataFrame 'data' with job data
data = pd.read_csv('working_best.csv')

# Feature extraction using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['Job Title'])

# Train KMeans model
n_clusters = 6  # Number of clusters for high, medium, and low demand
kmeans = KMeans(n_clusters=n_clusters, random_state=10)
kmeans.fit(X)

# Save the trained model
joblib.dump(kmeans, 'demand_kmeans_model.joblib')

# Load the saved model
kmeans = joblib.load('demand_kmeans_model.joblib')

# Assign cluster labels to job data
data['cluster'] = kmeans.labels_

# Count the number of job titles in each cluster
cluster_counts = data['cluster'].value_counts()

# Identify the cluster with the highest count (assuming the index corresponds to the cluster label)
high_demand_cluster = cluster_counts.idxmax()

# Group job titles by cluster and limit to first 5 unique jobs
clusters = {}
for cluster_label in range(n_clusters):
    cluster_data = data[data['cluster'] == cluster_label]
    job_titles = set()
    for job_title in cluster_data['Job Title']:
        if job_title not in job_titles:
            job_titles.add(job_title)
            if len(job_titles) == 10: 
                break
    clusters[cluster_label] = job_titles

# Display the high-demand cluster
print(f"The high-demand cluster is Cluster {high_demand_cluster}:")
for job_title in clusters[high_demand_cluster]:
    print(job_title)
print('\n')


/home/dell/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


The high-demand cluster is Cluster 0:
['social media coordinator']
['teacher']
['social worker']
['nurse practitioner']
['web developer']
['litigation attorney']
['software tester']
['legal assistant']
['qa analyst']
['hr generalist']


